In [6]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
import pandas as pd

In [7]:
username = "capstone_admin"
password = "Energy#2023"
host = "capstonedb.postgres.database.azure.com"
port = "5432"
database = "postgres"

In [8]:
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{database}")

In [74]:
df_Consumer = pd.DataFrame({
    'consumer_number': list(range(1, 2809))
})

df_Consumer.to_sql('consumer', engine, if_exists='append', index=False)

808

In [11]:
import pandas as pd

# Replace 'your_input_file.csv' with the actual file name
input_file = '../../Capstone/AnomalyDetection/Labelled_Data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

df = df.iloc[:, 1:-1]

# Melt the DataFrame to reshape it
df_melted = pd.melt(df, id_vars=['Consumer_Number'], var_name='Time', value_name='Consumption')
()

In [19]:
df_melted.loc[df_melted['Consumption'].notnull(), 'Consumer_Number'].unique()

array([   1,    2,    3, ..., 2806, 2807, 2808])

In [49]:
df_melted.tail()

,Consumer_Number,Time,Consumption
24598075,2804,12/31 24:00:00,0.620971
24598076,2805,12/31 24:00:00,0.615851
24598077,2806,12/31 24:00:00,0.627720
24598078,2807,12/31 24:00:00,0.653643
24598079,2808,12/31 24:00:00,0.630375


In [50]:
df_melted['Time'] = df_melted['Time'].str.strip()


In [52]:
df_melted['Time'] = '2022/' + df_melted['Time']

In [53]:
df_melted['Time'] = df_melted['Time'].str.replace(' 24:', ' 00:', regex=False)


In [55]:
df_melted['Time'] = pd.to_datetime(df_melted['Time'], format='%Y/%m/%d %H:%M:%S')

In [75]:
df_Consumption = df_melted.sort_values(by=['Consumer_Number', 'Time'])


In [77]:
df_Consumption.columns = [col.lower() for col in df_Consumption.columns]

In [78]:
df_Consumption.head()

,consumer_number,time,consumption
64584,1,2022-01-01 00:00:00,1.375976
0,1,2022-01-01 01:00:00,1.019364
2808,1,2022-01-01 02:00:00,0.887116
5616,1,2022-01-01 03:00:00,0.833463
8424,1,2022-01-01 04:00:00,0.818020


In [ ]:
df_Consumption.to_csv('../../Capstone/AnomalyDetection/FinalData.csv', index=False)

In [3]:
import pandas as pd

# Replace 'your_input_file.csv' with the actual file name
input_file = '../../Capstone/AnomalyDetection/Labelled_Data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

df_Fraud = df.iloc[:, [1, -1]]


In [4]:
df_Fraud.head()

,Consumer_Number,Fraud_Status
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [5]:
df_Fraud.columns = [col.lower() for col in df_Fraud.columns]

In [22]:
consumers_with_consumption = df_melted.loc[df_melted['Consumption'].notnull(), 'Consumer_Number'].unique()

df_Fraud = df_Fraud[df_Fraud['consumer_number'].isin(consumers_with_consumption)]

In [23]:
df_Fraud.to_sql('fraud', engine, if_exists='append', index=False)

805

In [43]:
import pandas as pd

# Define the range of values for area_code and city_code
area_code_range = range(1, 10)
city_code_range = range(1, 3)

num_records=2808

# Calculate the total number of combinations
total_combinations = len(area_code_range) * len(city_code_range)

# Calculate the number of times each combination should be repeated
repeats = (num_records // total_combinations)+1

# Create a list of combinations based on the desired distribution
combinations = [(area, city) for area in area_code_range for city in city_code_range] * repeats

# Create a DataFrame with the specified number of records and columns
df = pd.DataFrame(combinations[:2808], columns=['area_code', 'city_code'])

df=df.sample(frac=1).reset_index(drop=True)

df['consumer_number']=list(range(1, 2809))

In [44]:
df

,area_code,city_code,consumer_number
0,7,2,1
1,4,1,2
2,3,2,3
3,1,2,4
4,3,2,5
...,...,...,...
2803,2,2,2804
2804,3,1,2805
2805,4,2,2806
2806,6,2,2807


In [45]:
df.to_sql('temp', engine, if_exists='replace', index=False)

808